# **Librerie da installare per il db**

In [ ]:
import pandas as pd
import sqlite3
import requests
from io import StringIO

### **Codice per creare il db partendo da url o da file**

In [ ]:
# Definisco l'URL dei file che voglio recuperare
url_importanza_economica_csv = 'http://www.datiopen.it/export/csv/Importanza-economica-del-settore-della-pesca-per-regione.csv'

In [ ]:
# # Utilizzo la libreria requests per ricavarmi il contenuto dei file
response_importanza_economica_csv = requests.get(url_importanza_economica_csv)

In [ ]:
 # Se la richiesta non è andata a buon fine genero un'eccezione
response_importanza_economica_csv.raise_for_status()

In [ ]:
# Se la richiesta è andata a buon fine utilizzo il module di pandas per processare i dati
importanza_economica = pd.read_csv(StringIO(response_importanza_economica_csv.text), sep=';')

In [ ]:
# Patch per il nostro db
db_path = 'cartella_database/cartella_sqlite/pesca.db'

In [ ]:
# Connessione al db
conn = sqlite3.connect(db_path)

In [ ]:
# Creazione tabelle
importanza_economica.to_sql('importanza_economica', conn, index=False, if_exists='replace')

In [ ]:
# Chiusura connessione
conn.close()

### **Codice per le API**

In [2]:
%pip install pandas

     --------------------------------------- 11.6/11.6 MB 21.8 MB/s eta 0:00:00
     --------------------------------------- 16.5/16.5 MB 50.4 MB/s eta 0:00:00
     ------------------------------------- 505.5/505.5 kB 31.0 MB/s eta 0:00:00
     ---------------------------------------- 345.4/345.4 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from fastapi import FastAPI, HTTPException, Response
from pydantic import BaseModel, Field
import matplotlib.pyplot as plt
import sqlite3
from sqlite3 import Error
from io import BytesIO
import pandas as pd
from typing import Optional

In [ ]:
app = FastAPI()

In [ ]:
# Funzione che crea la connessione con il db
def create_connection():
    try:
        conn = sqlite3.connect('cartella_database/cartella_sqlite/pesca.db')

        return conn
    except Error as e:
        print(e)
        return None

In [26]:
@app.get("/get_media_percentuale_valore_aggiunto_pesca_piscicoltura/")
async def get_media_percentuale_valore_aggiunto_pesca_piscicoltura(year_a: Optional[int] = None,
                                                                   year_b: Optional[int] = None,
                                                                   plot_graph: Optional[bool] = False):
    # Controllo sugli anni
    if year_a is not None and year_b is not None:
        if year_a >= year_b:
            raise HTTPException(status_code=400, detail="Anno A deve essere più piccolo di Anno B")
        
    # Query da utilizzare
    conn = sqlite3.connect('cartella_database/cartella_sqlite/pesca.db')
    query = """
    SELECT *
    FROM importanza_economica
    """

    df = pd.read_sql_query(query, conn)


    # Definizione delle aree geografiche
    nord_ovest = ["Valle d'Aosta", "Piemonte", "Liguria", "Lombardia"]
    nord_est = ["Trentino-Alto Adige", "Veneto", "Friuli-Venezia Giulia", "Emilia-Romagna"]
    centro = ["Toscana", "Umbria", "Marche", "Lazio", "Abruzzo"]
    sud = ["Molise", "Campania", "Puglia", "Basilicata", "Calabria"]
    isole = ["Sicilia", "Sardegna"]

    # Lista delle aree
    aree = [nord_ovest, nord_est, centro, sud, isole]
    nomi_aree = ['Nord-Ovest', 'Nord-Est', 'Centro', 'Sud', 'Isole']


    # Inizializza una lista di dizionari per i risultati
    results = []
    for anno in df['Anno'].unique():
        # Inizializza il dizionario per salvare le medie percentuali per l'anno corrente
        medie_percentuali_anno = {'Anno': anno}
        
        # Itera per ogni area geografica
        for area, nome_area in zip(aree, nomi_aree):
            # Seleziona le righe del DataFrame corrispondenti all'anno e all'area geografica
            subset = df[(df['Anno'] == anno) & (df['Regione'].isin(area))]
            
            # Calcola la media percentuale del valore aggiunto della pesca e piscicoltura per l'area e l'anno correnti
            media_percentuale = subset['Percentuale_valore_aggiunto_pesca-piscicoltura-servizi'].mean()
            
            # Salva la media percentuale nel dizionario
            medie_percentuali_anno[nome_area] = media_percentuale
        
            # Crea un dizionario per il risultato corrente
            result = {
                'Anno': anno,
                'Aree': nome_area,
                'Media percentuale valore aggiunto pesca piscicoltura': media_percentuale
            }
            
            # Aggiungi il dizionario alla lista dei risultati
            results.append(result)

    # Converti la lista di dizionari in un DataFrame
    media_percentuale_valore_aggiunto_pesca_piscicoltura = pd.DataFrame(results)

    # Visualizza il DataFrame dei risultati finali
    display(media_percentuale_valore_aggiunto_pesca_piscicoltura)
    
    # Filtra i dati in base ai parametri year_a e year_b
    if year_a is not None and year_b is not None:
        filtered_data = media_percentuale_valore_aggiunto_pesca_piscicoltura[
            (media_percentuale_valore_aggiunto_pesca_piscicoltura['Anno'] >= year_a) &
            (media_percentuale_valore_aggiunto_pesca_piscicoltura['Anno'] <= year_b)
        ]
    elif year_a is not None and year_b is None:
        filtered_data = media_percentuale_valore_aggiunto_pesca_piscicoltura[
            (media_percentuale_valore_aggiunto_pesca_piscicoltura['Anno'] >= year_a)
        ]
    elif year_a is None and year_b is not None:
        filtered_data = media_percentuale_valore_aggiunto_pesca_piscicoltura[
            (media_percentuale_valore_aggiunto_pesca_piscicoltura['Anno'] <= year_b)
        ]
    else:
        filtered_data = media_percentuale_valore_aggiunto_pesca_piscicoltura
        
    # Chiudi la connessione al database
    conn.close()
    
    if plot_graph:
        plt.figure(figsize=(10, 6))
        for area in nomi_aree:
            plt.plot(filtered_data['Anno'], filtered_data[area], marker='o', label=area)
        plt.xlabel('Anno')
        plt.ylabel('Media percentuale valore aggiunto pesca piscicoltura')
        plt.title('Media percentuale valore aggiunto pesca piscicoltura per area geografica')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Salva il grafico in un buffer BytesIO
        buffer = BytesIO()
        plt.savefig(buffer, format='png')
        buffer.seek(0)

        # Ritorna il grafico come risposta HTTP
        return Response(content=buffer.getvalue(), media_type='image/png')
    
    return filtered_data 


    

,Anno,Regione,Percentuale_valore_aggiunto_pesca-piscicoltura-servizi
0,2000,Piemonte,0.01
1,2000,Valle d'Aosta,0.01
2,2000,Lombardia,0.01
3,2000,Trentino-Alto Adige,0.02
4,2000,Veneto,0.16
...,...,...,...
235,2011,Puglia,0.40
236,2011,Basilicata,0.01
237,2011,Calabria,0.13
238,2011,Sicilia,0.36


array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011])

,Anno,Aree,Media percentuale valore aggiunto pesca pisciocoltura


,Anno,Aree,Media percentuale valore aggiunto pesca piscicoltura
0,2000,Nord-Ovest,0.0350
1,2000,Nord-Est,0.1475
2,2000,Centro,0.1580
3,2000,Sud,0.1820
4,2000,Isole,0.3250
5,2001,Nord-Ovest,0.0400
6,2001,Nord-Est,0.1350
7,2001,Centro,0.1520
8,2001,Sud,0.1720
9,2001,Isole,0.3300
